In [1]:
import geopandas as gpd
import pandas as pd
import folium
from folium import Element



In [2]:

# --- CARGAR SHAPEFILE ---
ruta_shapefile = 'C:/Users/mauri/OneDrive - Departamento Nacional de Planeacion/Documentos/0. Resources/2. MGN DANE/MGN2023_MPIO_POLITICO/MGN_ADM_MPIO_GRAFICO.shp'
gdf_municipios = gpd.read_file(ruta_shapefile)

ruta_shapefile2 = 'C:/Users/mauri/OneDrive - Departamento Nacional de Planeacion/Documentos/0. Resources/2. MGN DANE/MGN2023_DPTO_POLITICO/MGN_ADM_DPTO_POLITICO.shp'
gdf_departamentos = gpd.read_file(ruta_shapefile2)


c:\ProgramData\anaconda3\envs\ent_31015_maps\lib\site-packages\pyogrio\core.py:35: RuntimeWarning: Could not detect GDAL data files.  Set GDAL_DATA environment variable to the correct path.
  _init_gdal_data()


In [3]:

# --- CARGAR ENTIDADES ---
ruta_entidades = 'C:/Users/mauri/OneDrive - Departamento Nacional de Planeacion/Documentos/DNP-760-2024/Informes/0. Mapas py/input/Todos los Mpios.xlsx'
df_entidades = pd.read_excel(ruta_entidades)

def clasificar_estado(valor):
    if pd.isnull(valor):
        return 'No disponible'
    elif valor == 0:
        return 'No iniciado'
    elif 0 < valor < 0.5:
        return 'Inicial'
    elif valor == 0.5:
        return 'Intermedio'
    elif 0.5 < valor < 1:
        return 'Avanzado'
    elif valor == 1:
        return 'Finalizado'
    else:
        return 'No disponible'

df_entidades['Estado'] = df_entidades['%Sistematización'].apply(clasificar_estado)



MAPA - UNO POR DEPARTAMENTO

In [12]:
# --- FILTRAR SOLO VALLE DEL CAUCA (76) ---
gdf_entidad_interes = gdf_municipios[gdf_municipios['dpto_ccdgo'] == '05'].copy()


In [18]:

# --- UNIFICAR CÓDIGOS ---
gdf_entidad_interes['mpio_cdpmp'] = gdf_entidad_interes['mpio_cdpmp'].astype(str).str.zfill(5)
df_entidades['COD_MUNI'] = df_entidades['COD_MUNI'].astype(str).str.zfill(5)

# --- MERGE ---
gdf_entidad_interes = gdf_entidad_interes.merge(
    df_entidades[['COD_MUNI', 'Estado', '%Sistematización']],
    left_on='mpio_cdpmp', right_on='COD_MUNI', how='left'
)
gdf_entidad_interes = gdf_entidad_interes[~gdf_entidad_interes['geometry'].isnull()]

# --- FORMATEAR PORCENTAJE ---
gdf_entidad_interes['SisPT Avance %'] = gdf_entidad_interes['%Sistematización'].apply(
    lambda x: f"{round(x * 100, 1)}%" if pd.notnull(x) else "No disponible"
)

# --- COLORES ---
color_map = {
    'Finalizado': 'green',
    'Intermedio': 'orange',
    'Avanzado': 'blue',
    'No iniciado': 'red',
    'Inicial': 'lightblue'
}

def style_function(feature):
    estado = feature['properties'].get('Estado')# Obtiene el valor de la propiedad 'Estado' del municipio actual (cada feature es un municipio)
    # Devuelve un diccionario con los estilos visuales para ese municipio en el mapa
    return {
        'fillColor': color_map.get(estado, 'lightgray'),# Color de relleno del polígono (según el estado); si no existe el estado, se usa 'lightgray'
        'color': 'black',# Color del borde del polígono (negro)
        'weight': 1,# Grosor de la línea del borde
        'fillOpacity': 0.7# Opacidad del relleno (0: transparente, 1: completamente opaco)
    }

# --- CREAR MAPA ---
m = folium.Map(location=[3.5, -76.5], zoom_start=8, tiles='CartoDB positron')  # Crea el mapa centrado en el Valle del Cauca con un nivel de zoom inicial




# --- AGREGAR CAPA DE DEPARTAMENTO (DEBAJO DE LOS MUNICIPIOS) ---
folium.GeoJson(
    gdf_departamentos,
    name='Departamento',
    style_function=lambda feature: {
        'fillColor': '#f0f0f0',  # gris claro
        'color': 'gray',         # borde gris
        'weight': 1,
        'fillOpacity': 0.3
    }
).add_to(m)


# --- CAPA GEOJSON MPIOS---
folium.GeoJson(                                  # Crea una capa de polígonos (GeoJSON) sobre el mapa
    gdf_entidad_interes,                                   # GeoDataFrame con geometría y atributos de los municipios del Valle
    name='Estado entidades',                     # Nombre de la capa (útil si se agrega un control de capas)
    style_function=style_function,               # Función que define el estilo de relleno de cada municipio según su Estado
    tooltip=folium.GeoJsonTooltip(               # Tooltip que aparece al pasar el cursor sobre un municipio
        fields=['mpio_cnmbr', 'Estado', 'SisPT Avance %'],  # Campos a mostrar en el tooltip
        aliases=['Municipio:', 'Estado:', 'Avance SisPT:'], # Nombres amigables para los campos del tooltip
        localize=True                            # Permite formatear automáticamente números/localización si aplica
    )
).add_to(m)                                      # Añade la capa GeoJSON al mapa


# --- LEYENDA HTML ---
legend_html = """
<div style="
    position: fixed; 
    bottom: 50px; left: 50px; width: 200px; 
    z-index:9999; font-size:14px;
    background-color: white;
    border:2px solid grey;
    padding: 10px;
    border-radius: 5px;
">
<b>Estado de Sistematización</b><br>
<i style="background:green; width:15px; height:15px; float:left; margin-right:8px;"></i> Finalizado<br>
<i style="background:orange; width:15px; height:15px; float:left; margin-right:8px;"></i> Intermedio<br>
<i style="background:blue; width:15px; height:15px; float:left; margin-right:8px;"></i> Avanzado<br>
<i style="background:red; width:15px; height:15px; float:left; margin-right:8px;"></i> No iniciado<br>
<i style="background:lightblue; width:15px; height:15px; float:left; margin-right:8px;"></i> Inicial<br>
</div>
"""
m.get_root().html.add_child(Element(legend_html))

# --- ETIQUETAS ESTÁTICAS DE MUNICIPIOS ---
for _, row in gdf_entidad_interes.iterrows():
    if pd.notnull(row['geometry']) and pd.notnull(row['mpio_cnmbr']):
        centroid = row['geometry'].centroid
        folium.map.Marker(
            [centroid.y, centroid.x],
            icon=folium.DivIcon(
                html=f"""<div style="font-size:10px; color:black; text-align:center;">{row['mpio_cnmbr'].title()}</div>"""
            )
        ).add_to(m)

# --- GUARDAR MAPA ---

from datetime import date

# Obtener la fecha actual en formato AAAA-MM-DD
fecha_actual = date.today().strftime('%Y-%m-%d')


nombre_departamento = df_entidades['DPTO'].dropna().unique()[0]

# Construir el nombre del archivo con la fecha
nombre_archivo = f"{fecha_actual} SisPT {nombre_departamento}_estado2.html"

# Ruta completa
ruta_salida = f'C:/Users/mauri/OneDrive - Departamento Nacional de Planeacion/Documentos/DNP-760-2024/Informes/0. Mapas py/output/{nombre_archivo}'

# Guardar el mapa
m.save(ruta_salida)

MAPA COMPLETO COLOMBIA - MUNICIPIOS SELECCIONADOS

In [75]:
gdf_entidad_interes = gdf_municipios.copy()

In [76]:
df_entidades_filt = df_entidades[df_entidades['Elecciones_atipicas']== 1].copy()

In [77]:
cols_binarias = [
    'Registro Problematicas', 'Formulación Objetivo Principal',
    'Formulación Objetivos Específicios', 'Formulación Líneas Estratégicas',
    'Formulación Indicadores Producto', 'Cargue Documento PISCC y Acta',
    'Formulación Actividades'
]

In [78]:
# Convertir todas las columnas binarias (menos la de 3 categorías) con map
for col in cols_binarias:
    if col != 'Cargue Documento PISCC y Acta':
        df_entidades_filt[col] = pd.to_numeric(df_entidades_filt[col], errors='coerce')
        df_entidades_filt[col] = df_entidades_filt[col].map({1: 'Sí', 0: 'No'}).fillna('No disponible')

# Manejar el caso especial de 0, 1, 2
df_entidades_filt['Cargue Documento PISCC y Acta'] = pd.to_numeric(
    df_entidades_filt['Cargue Documento PISCC y Acta'], errors='coerce'
)
df_entidades_filt['Cargue Documento PISCC y Acta'] = df_entidades_filt['Cargue Documento PISCC y Acta'].apply(
    lambda x: 'Sí' if x == 2 else ('No' if x in [0, 1] else 'No disponible')
)



In [ ]:

# --- UNIFICAR CÓDIGOS ---
gdf_entidad_interes['mpio_cdpmp'] = gdf_entidad_interes['mpio_cdpmp'].astype(str).str.zfill(5)
df_entidades_filt['COD_MUNI'] = df_entidades_filt['COD_MUNI'].astype(str).str.zfill(5)

# --- MERGE ---
gdf_entidad_interes = gdf_entidad_interes.merge(
    df_entidades_filt[['COD_MUNI', 'Estado', '%Sistematización','Registro Problematicas', 'Formulación Objetivo Principal',
    'Formulación Objetivos Específicios', 'Formulación Líneas Estratégicas',
    'Formulación Indicadores Producto', 'Cargue Documento PISCC y Acta',
    'Formulación Actividades']],
    left_on='mpio_cdpmp', right_on='COD_MUNI', how='right'
) # Aquí se hacer merge por derecha 'right' para mantener todos los municipios de elecciones atípicas

gdf_entidad_interes = gdf_entidad_interes[~gdf_entidad_interes['geometry'].isnull()]

# --- FORMATEAR PORCENTAJE ---
gdf_entidad_interes['SisPT Avance %'] = gdf_entidad_interes['%Sistematización'].apply(
    lambda x: f"{round(x * 100, 1)}%" if pd.notnull(x) else "No disponible"
)

# --- COLORES ---
color_map = {
    'Finalizado': 'green',
    'Intermedio': 'orange',
    'Avanzado': 'blue',
    'No iniciado': 'red',
    'Inicial': 'lightblue'
}

def style_function(feature):
    estado = feature['properties'].get('Estado')# Obtiene el valor de la propiedad 'Estado' del municipio actual (cada feature es un municipio)
    # Devuelve un diccionario con los estilos visuales para ese municipio en el mapa
    return {
        'fillColor': color_map.get(estado, 'lightgray'),# Color de relleno del polígono (según el estado); si no existe el estado, se usa 'lightgray'
        'color': 'black',# Color del borde del polígono (negro)
        'weight': 1,# Grosor de la línea del borde
        'fillOpacity': 0.7# Opacidad del relleno (0: transparente, 1: completamente opaco)
    }

# --- CREAR MAPA ---
m = folium.Map(location=[3.5, -76.5], zoom_start=8, tiles='CartoDB positron')  # Crea el mapa centrado en el Valle del Cauca con un nivel de zoom inicial




# --- AGREGAR CAPA DE DEPARTAMENTO (DEBAJO DE LOS MUNICIPIOS) ---
folium.GeoJson(
    gdf_departamentos,
    name='Departamento',
    style_function=lambda feature: {
        'fillColor': '#f0f0f0',  # gris claro
        'color': 'gray',         # borde gris
        'weight': 1,
        'fillOpacity': 0.3
    }
).add_to(m)


# --- CAPA GEOJSON MPIOS---
folium.GeoJson(                                  # Crea una capa de polígonos (GeoJSON) sobre el mapa
    gdf_entidad_interes,                                   # GeoDataFrame con geometría y atributos de los municipios del Valle
    name='Estado entidades',                     # Nombre de la capa (útil si se agrega un control de capas)
    style_function=style_function,               # Función que define el estilo de relleno de cada municipio según su Estado
    tooltip=folium.GeoJsonTooltip(
    fields=[
        'mpio_cnmbr', 'Estado', 'SisPT Avance %',
        'Registro Problematicas', 'Formulación Objetivo Principal',
        'Formulación Objetivos Específicios', 'Formulación Líneas Estratégicas',
        'Formulación Indicadores Producto', 'Cargue Documento PISCC y Acta',
        'Formulación Actividades'
    ],
    aliases=[
        'Municipio:', 'Estado:', 'Avance SisPT:',
        'Registro de Problemáticas:', 'Objetivo Principal:',
        'Objetivos Específicos:', 'Líneas Estratégicas:',
        'Indicadores de Producto:', 'Documento PISCC y Acta:',
        'Actividades:'
    ],
    localize=True
)
).add_to(m)                                      # Añade la capa GeoJSON al mapa


# --- LEYENDA HTML ---
legend_html = """
<div style="
    position: fixed; 
    bottom: 50px; left: 50px; width: 200px; 
    z-index:9999; font-size:14px;
    background-color: white;
    border:2px solid grey;
    padding: 10px;
    border-radius: 5px;
">
<b>Estado de Sistematización</b><br>
<i style="background:green; width:15px; height:15px; float:left; margin-right:8px;"></i> Finalizado<br>
<i style="background:orange; width:15px; height:15px; float:left; margin-right:8px;"></i> Intermedio<br>
<i style="background:blue; width:15px; height:15px; float:left; margin-right:8px;"></i> Avanzado<br>
<i style="background:red; width:15px; height:15px; float:left; margin-right:8px;"></i> No iniciado<br>
<i style="background:lightblue; width:15px; height:15px; float:left; margin-right:8px;"></i> Inicial<br>
</div>
"""
m.get_root().html.add_child(Element(legend_html))

# --- ETIQUETAS ESTÁTICAS DE MUNICIPIOS ---
for _, row in gdf_entidad_interes.iterrows():
    if pd.notnull(row['geometry']) and pd.notnull(row['mpio_cnmbr']):
        centroid = row['geometry'].centroid
        folium.map.Marker(
            [centroid.y, centroid.x],
            icon=folium.DivIcon(
                html=f"""<div style="font-size:10px; color:black; text-align:center;">{row['mpio_cnmbr'].title()}</div>"""
            )
        ).add_to(m)

# --- GUARDAR MAPA ---

from datetime import date

# Obtener la fecha actual en formato AAAA-MM-DD
fecha_actual = date.today().strftime('%Y-%m-%d')



# Construir el nombre del archivo con la fecha
nombre_archivo = f"{fecha_actual} SisPT Elecciones Atípicas.html"

# Ruta completa
ruta_salida = f'C:/Users/mauri/OneDrive - Departamento Nacional de Planeacion/Documentos/DNP-760-2024/Informes/0. Mapas py/output/{nombre_archivo}'

# Guardar el mapa
m.save(ruta_salida)

MAPA - MULTIPLES DEPARTAMENTOS

In [4]:
departamentos_deseados = ['13', '18', '19', '25', '41', '50', '52', '54', '76', '86']


In [5]:
df_entidades['COD_DPTO'] = df_entidades['COD_DPTO'].astype(str).str.zfill(2)
df_entidades_filt = df_entidades[df_entidades['COD_DPTO'].isin(departamentos_deseados)].copy()

In [6]:
cols_binarias = [
    'Registro Problematicas', 'Formulación Objetivo Principal',
    'Formulación Objetivos Específicios', 'Formulación Líneas Estratégicas',
    'Formulación Indicadores Producto', 'Cargue Documento PISCC y Acta',
    'Formulación Actividades'
]

In [7]:
# Convertir todas las columnas binarias (menos la de 3 categorías) con map
for col in cols_binarias:
    if col != 'Cargue Documento PISCC y Acta':
        df_entidades_filt[col] = pd.to_numeric(df_entidades_filt[col], errors='coerce')
        df_entidades_filt[col] = df_entidades_filt[col].map({1: 'Sí', 0: 'No'}).fillna('No disponible')

# Manejar el caso especial de 0, 1, 2
df_entidades_filt['Cargue Documento PISCC y Acta'] = pd.to_numeric(
    df_entidades_filt['Cargue Documento PISCC y Acta'], errors='coerce'
)
df_entidades_filt['Cargue Documento PISCC y Acta'] = df_entidades_filt['Cargue Documento PISCC y Acta'].apply(
    lambda x: 'Sí' if x == 2 else ('No' if x in [0, 1] else 'No disponible')
)


In [8]:

for cod_dpto in departamentos_deseados:
    # Filtrar shapefile por departamento
    gdf_entidad_interes = gdf_municipios[gdf_municipios['dpto_ccdgo'] == cod_dpto].copy()
    
    # --- UNIFICAR CÓDIGOS ---
    gdf_entidad_interes['mpio_cdpmp'] = gdf_entidad_interes['mpio_cdpmp'].astype(str).str.zfill(5)
    df_entidades_filt['COD_MUNI'] = df_entidades_filt['COD_MUNI'].astype(str).str.zfill(5)

    # --- MERGE ---
    gdf_entidad_interes = gdf_entidad_interes.merge(
        df_entidades_filt[['COD_MUNI', 'Estado',  '%Sistematización','Registro Problematicas', 'Formulación Objetivo Principal',
    'Formulación Objetivos Específicios', 'Formulación Líneas Estratégicas',
    'Formulación Indicadores Producto', 'Cargue Documento PISCC y Acta',
    'Formulación Actividades']],
        left_on='mpio_cdpmp', right_on='COD_MUNI', how='left'
    )
    gdf_entidad_interes = gdf_entidad_interes[~gdf_entidad_interes['geometry'].isnull()]

    # --- FORMATEAR PORCENTAJE ---
    gdf_entidad_interes['SisPT Avance %'] = gdf_entidad_interes['%Sistematización'].apply(
        lambda x: f"{round(x * 100, 1)}%" if pd.notnull(x) else "No disponible"
    )

    # --- COLORES ---
    color_map = {
        'Finalizado': 'green',
        'Intermedio': 'orange',
        'Avanzado': 'blue',
        'No iniciado': 'red',
        'Inicial': 'lightblue'
    }

    def style_function(feature):
        estado = feature['properties'].get('Estado')# Obtiene el valor de la propiedad 'Estado' del municipio actual (cada feature es un municipio)
        # Devuelve un diccionario con los estilos visuales para ese municipio en el mapa
        return {
            'fillColor': color_map.get(estado, 'lightgray'),# Color de relleno del polígono (según el estado); si no existe el estado, se usa 'lightgray'
            'color': 'black',# Color del borde del polígono (negro)
            'weight': 1,# Grosor de la línea del borde
            'fillOpacity': 0.7# Opacidad del relleno (0: transparente, 1: completamente opaco)
        }

    # --- CREAR MAPA ---
    m = folium.Map(location=[3.5, -76.5], zoom_start=8, tiles='CartoDB positron')  # Crea el mapa centrado en el Valle del Cauca con un nivel de zoom inicial




    # --- AGREGAR CAPA DE DEPARTAMENTO (DEBAJO DE LOS MUNICIPIOS) ---
    folium.GeoJson(
        gdf_departamentos,
        name='Departamento',
        style_function=lambda feature: {
            'fillColor': '#f0f0f0',  # gris claro
            'color': 'gray',         # borde gris
            'weight': 1,
            'fillOpacity': 0.3
        }
    ).add_to(m)


    # --- CAPA GEOJSON MPIOS---
    folium.GeoJson(                                  # Crea una capa de polígonos (GeoJSON) sobre el mapa
        gdf_entidad_interes,                                   # GeoDataFrame con geometría y atributos de los municipios del Valle
        name='Estado entidades',                     # Nombre de la capa (útil si se agrega un control de capas)
        style_function=style_function,               # Función que define el estilo de relleno de cada municipio según su Estado
        tooltip=folium.GeoJsonTooltip(
        fields=[
            'mpio_cnmbr', 'Estado', 'SisPT Avance %',
            'Registro Problematicas', 'Formulación Objetivo Principal',
            'Formulación Objetivos Específicios', 'Formulación Líneas Estratégicas',
            'Formulación Indicadores Producto', 'Cargue Documento PISCC y Acta',
            'Formulación Actividades'
        ],
        aliases=[
            'Municipio:', 'Estado:', 'Avance SisPT:',
            'Registro de Problemáticas:', 'Objetivo Principal:',
            'Objetivos Específicos:', 'Líneas Estratégicas:',
            'Indicadores de Producto:', 'Documento PISCC y Acta:',
            'Actividades:'
        ],
        localize=True
    )
    ).add_to(m)                                      # Añade la capa GeoJSON al mapa


    # --- LEYENDA HTML ---
    legend_html = """
    <div style="
        position: fixed; 
        bottom: 50px; left: 50px; width: 200px; 
        z-index:9999; font-size:14px;
        background-color: white;
        border:2px solid grey;
        padding: 10px;
        border-radius: 5px;
    ">
    <b>Estado de Sistematización</b><br>
    <i style="background:green; width:15px; height:15px; float:left; margin-right:8px;"></i> Finalizado<br>
    <i style="background:orange; width:15px; height:15px; float:left; margin-right:8px;"></i> Intermedio<br>
    <i style="background:blue; width:15px; height:15px; float:left; margin-right:8px;"></i> Avanzado<br>
    <i style="background:red; width:15px; height:15px; float:left; margin-right:8px;"></i> No iniciado<br>
    <i style="background:lightblue; width:15px; height:15px; float:left; margin-right:8px;"></i> Inicial<br>
    </div>
    """
    m.get_root().html.add_child(Element(legend_html))

    # --- ETIQUETAS ESTÁTICAS DE MUNICIPIOS ---
    for _, row in gdf_entidad_interes.iterrows():
        if pd.notnull(row['geometry']) and pd.notnull(row['mpio_cnmbr']):
            centroid = row['geometry'].centroid
            folium.map.Marker(
                [centroid.y, centroid.x],
                icon=folium.DivIcon(
                    html=f"""<div style="font-size:10px; color:black; text-align:center;">{row['mpio_cnmbr'].title()}</div>"""
                )
            ).add_to(m)

    # --- GUARDAR MAPA ---

    from datetime import date

    # Obtener la fecha actual en formato AAAA-MM-DD
    fecha_actual = date.today().strftime('%Y-%m-%d')

    # Obtener nombre del departamento desde el shapefile (opcional)
    nombre_departamento = gdf_entidad_interes['dpto_cnmbr'].dropna().unique()[0].title()

    # Guardar el mapa con el nombre del departamento
    nombre_archivo = f"{fecha_actual} SisPT {nombre_departamento}_estado.html"
    ruta_salida = f'C:/Users/mauri/OneDrive - Departamento Nacional de Planeacion/Documentos/DNP-760-2024/Informes/0. Mapas py/output/{nombre_archivo}'
    m.save(ruta_salida)